# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

Res:

一种计算方式是没有log的；另一种计算方式是有log的，log的底数10是可以调整的，TF用10为底数，那么IDF也应该用10作为底数。

$ Term Frequency ： TF(t, d) = \log_{10} {count(t,d) + 1} $

其中 count(t,d) 是一个句子（文档）中t这个词出现的次数

$ Inverse Document Frequency ： IDF(t, d) = \log_{10} { \frac{N}{df_t} } $

其中 N 是文档总数，$df_t$ 是t这个词出现的文档数

$ weighted value ： TF-IDF = TF(t, d) \times IDF(t, d) $


#### 2. LDA算法的基本假设是什么？

Res:
 
LDA (Latent Dirichlet Allocation 隐藏迪利克雷（多项式）分布) 的基本假设是： 文档到主题服从多项式分布，
主题到词服从多项式分布。

#### 3. 在TextRank算法中构建图的权重是如何得到的？

Res:
1. 去掉文中的停用词
2. 使用滑动窗口的方式去构建图
3. 根据上面的步骤建立的图来计算权重
刚开始：让权重$ WS(V_i) = \frac{1}{N} $

设置根据下面的计算公式迭代，直到所有数值都稳定下来，可得到权重。

$$$$$$ WS(V_i) = (1 - d) + d * \sum_{j \in In(V_i)} { \frac {w_{ji} \times WS(V_j)}{\sum_{V_k \in Out(V_j)}{w_{jk}}} }  $$$$$$

其中$In(V_i)$表示所有指向$V_i$点的点；
$Out(V_j)$表示$V_j$点指向别的点的数量；
$WS(V_j)$表示连接的那个点的数值；
$w_{ji}$ 和 $w_{jk}$ 表示第j点和第i点以及第j点和第k点的权重值,通常用词与词之间的相似性表示，用余弦相似度来计算；
 $d$是个经验数值，通常取0.85
 
 计算得出的权重值越大表示这个词越重要。

#### 4. 什么是命名实体识别？ 有什么应用场景？

Res:

命名实体识别(NER)是指识别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词等。简单的讲，
就是识别自然文本中的实体指称的边界和类别。
应用场景:
问答系统，从问题里提取实体名；情感分析里分离出关键性的实体名

#### 5.NLP主要有哪几类任务 ？
1. 情感分析
2. 文本分类
3. 文本生成
4. 问答系统
5. 文本摘要


### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)


In [74]:
text = """
杨幂魏大勋恋情实锤又来了？网友：不官宣是为了以后更好的相处

作为“顶流明星”的杨幂，她的爱情故事自然会被不少网友密切关心着。和魏大勋的那些故事已经在“网络江湖”上流传了一些日子了，如今又传出魏大勋拍完戏后和杨幂一同乘车回酒店，在酒店门口吃完了助理帮两人买的冰淇淋后一同上了楼。

这样的消息当然很快就登上了各大网站的热搜和头条，只不过这一次网友们的评论显然要比以前温柔了不少，不少网友分析出几大要点后坦言：“俗话说女大三抱金砖，是真爱就开开心心在一起吧。”也有网友认为：“不官宣或许是为了以后更好的相处。”


请点击输入图片
从去年下半年杨幂和魏大勋传出爱情故事以来，就不断地成为了网友们茶余饭后的谈资。双方的粉丝各自认为对方不太符合自己心中偶像的“择偶标准”，甚至还一度出现了“脱粉”的事情。

其实这何尝不是网友们的“杞人忧天”，偶像也和大家一样有着自己对爱情的向往和追逐，也有着和普通人一样的“七情六欲”。有网友总结了几点后认为两人真可以很好地在一起。


一、“女大三抱金砖。”

本来是民间的俗语，但是被网友们一细细分析，就成为了所谓的“名正言顺”了。杨幂和魏大勋都是上世纪八零后出生的明星。杨幂是86年的，而魏大勋是89年的，两人的年纪虽然相差3岁，但是终究是一个时代的人。

和刘恺威相比，两人自然没有所谓很大的“代沟”，相处起来也应该会更容易不少。尤其是符合“女大三抱金砖”这句俗语，这也就让不少杨幂的粉丝“劝嫁”魏大勋了。


二、两人性格互补，更容易相处。

过来人都知道，两个人要在一起，最好就是要性格互补。而杨幂和魏大勋显然就是这样的“一对”。杨幂的性格更有北方女孩的“大咧咧”，从以前不少消息中都可以看出，杨幂是那种该笑就放声大笑，喜欢就义无反顾的人，某种角度上来说，杨幂有着的就是“真性情”。

2016年杨幂参加《真正男子汉》中就能够看得出来，杨幂不但是“学霸”，有着“高情商”，更是能和队友和教官都打成一片，有网友说这是她爱“撩”，但是不得不说这个节目的确也让杨幂“圈粉”不少。


说到魏大勋参加的综艺节目，很多网友都会想到他参加的《向往的生活》。在黄磊和何炅两位“前辈”的面前，魏大勋竟然一五一十地对着大家吐露着心声。他不但被前辈们套出了“30岁了很孤独，想找女朋友。”还成为了“《向往的生活》史上嘉宾第一个有语录的人。”

在赞赏黄磊时魏大勋这样说到：“优秀的人就不用走捷径了，优秀的人，他只要走起来就是捷径。”一时间让黄磊开心不已。于是有网友这样说到：“杨幂和魏大勋一个精明美丽一个憨厚淳朴，两人互补的性格在一起也容易相处。”


再说回这一次魏大勋和杨幂的事。4月12日是魏大勋生日，他在一条恋爱相关话题的微博下评论称：“4月12日，农历三月二十，希望今天生日的小伙伴可以和这辈子最喜欢的人在一起。”

这让不少网友认为魏大勋已经在暗恋自己跟喜欢的人在一起了，后来他还在评论区写道：“我19了”，而魏大勋的19岁，正是他在个人社交网站上表白过杨幂的年龄。


魏大勋对杨幂一直是“直言不讳”的喜欢，在不少综艺节目中都公开表白过。作为杨幂的“迷弟”，能过用这么多年追到偶像也算是一段佳话。

而双方的粉丝至今依然“严守”一开始的“信条”。杨幂的粉丝大多认为魏大勋颜值一般，而魏大勋的粉丝认为杨幂不但年纪大了魏大勋3岁，而且还“结婚生过孩子”。

两人之所以一直不“官宣”，或许也是希望以后更好的相处。那么这一次网曝的杨幂和魏大勋一起吃冰淇淋同住一酒店是不是两人关系“实锤”了呢？评论区继续聊聊吧。#杨幂魏大勋恋情实锤来了#


原创作品，喜欢请关注点赞。
"""

In [75]:
import os
import math
import jieba
import jieba.posseg
import networkx as nx
import codecs
import numpy as np

sentence_delimiters = ['?', '!', ';', '？', '！', '。', '；', '……', '…', '\n']
allow_speech_tags = ['an', 'i', 'j', 'l', 'n', 'nr', 'nrfg', 'ns', 'nt', 'nz', 't', 'v', 'vd', 'vn', 'eng']

In [76]:
#### 数据处理

####### 获取停用词
with open('../../DataSets/StopWords/Chinese/baidu_stopwords.txt','r',encoding='utf-8') as f:
    stop_words = [w.strip() for w in f.readlines()]
print('stop_words:', stop_words)

####### 分割句子
temp = [text]
for sep in sentence_delimiters:
    _text, temp = temp, []
    for seq in _text:
        temp += seq.split(sep)
sentences = [s.strip() for s in temp if len(s.strip()) > 0]
print('sentences:', sentences)

####### 分割成词
sentence_words = []
words = []
for sentence in sentences:
    jieba_result = list(jieba.posseg.cut(sentence))
    # 去除特殊符号 和 停用词
    word_list = [w.word.strip() for w in jieba_result 
                 if w.flag!='x' and w.word not in stop_words]
    words += word_list
    sentence_words.append(word_list)
print('words:', words)
print('sentence_words:', sentence_words)

stop_words: ['--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', "c'mon", 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain'

In [77]:
# 载入词向量
from gensim.models.word2vec import Word2Vec

word2VecModelFilePath = '../../DataSets\\Word2Vect\\xiejunjie_300_jieba\\wiki_han_word2vec_300维度.model'
Word2Vec_model = Word2Vec.load(word2VecModelFilePath)
Word2Vec_model['元帅']

C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-0.55147904,  0.05349015,  0.03995734, -0.1906048 , -0.2602298 ,
       -0.24075955, -0.0318087 ,  0.08650991,  0.38216746,  0.29373014,
       -0.42963234,  0.24906076,  0.526658  ,  0.11335317, -0.41380724,
        0.41810152,  0.34957063,  0.84539974, -0.32579637, -0.4241009 ,
       -0.0800854 , -0.3182321 , -0.15737769,  0.0641676 , -0.27024806,
        0.3087098 , -0.25946257,  0.29049155,  0.06181574, -0.35344124,
       -0.23557526, -0.34404266, -0.73496145, -0.6755665 ,  0.14357874,
       -0.46304873, -0.75077534, -0.4967365 ,  0.3866933 , -0.15416117,
        0.11419416, -0.5907989 ,  0.28164297,  0.08234562, -0.40139848,
        0.24460107,  0.4046515 ,  0.7396789 , -0.37050426,  0.05524458,
        0.42265978,  0.39167327, -0.10281888, -0.4250107 ,  0.37307784,
        0.34979337, -0.04144141,  0.62717354,  0.31276637, -0.7935658 ,
       -0.9111815 , -0.43097234, -0.4870675 , -0.6691357 ,  0.46876422,
       -0.574207  , -0.2173542 ,  0.29166907, -0.20126739, -0.52

In [93]:
# 计算词向量的余弦相似性
def cosine_similarity(x, y, norm=False):
    """ 计算两个向量x和y的余弦相似度 """
    assert len(x) == len(y), "len(x) != len(y)"
    zero_list = [0] * len(x)
    if list(x) == zero_list or list(y) == zero_list:
        return float(1) if list(x) == list(y) else float(0)

    # method 1
    res = np.array([[x[i] * y[i], x[i] * x[i], y[i] * y[i]] for i in range(len(x))])
    cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))
    return 0.5 * cos + 0.5 if norm else cos  # 归一化到[0, 1]区间内

# 建立图链接
wordSet = list(set(words))
num_words = len(wordSet)
print('num_words', num_words)
graph = {} #np.zeros((num_words, num_words))

word_connections = {}
win_size = 5

def add_wordToConnections(word_connections, words):
    # print('add words', words)
    for word in words:
        if word not in word_connections:
            word_connections[word] = set()
    for word in words:
        for word1 in words:
            if word != word1:
                word_connections[word].add(word1)
                word_connections[word1].add(word)
    
for i, wordList in enumerate(sentence_words):
    if win_size >= len(wordList):
        add_wordToConnections(word_connections, wordList)
    else:
        for j in range(win_size, len(wordList) + 1):
            add_wordToConnections(word_connections, wordList[j - win_size : j])
print('word_connections', word_connections)            

# 通过词向量相似度确定图上边的权重
for x in wordSet:
    for y in wordSet:
        if x in Word2Vec_model and y in Word2Vec_model:
            similarity = cosine_similarity( Word2Vec_model[x], Word2Vec_model[y] )
            graph[y, x] = graph[x, y] = similarity
        else:
            graph[y, x] = graph[x, y] = -1
        
# 迭代计算每个词的权重
weight_words = np.ones(num_words) / num_words
print('weight_words', weight_words)

num_words 259
word_connections {'杨幂': {'不少', '乘车', '都', '世纪', '美丽', '顶流', '更', '恋情', '情商', '爱情故事', '来说', '精明', '迷弟', '某种', '事', '下半年', '一同', '大多', '颜值', '传出', '不', '魏大勋', '2016', '多年', '一对', '冰淇淋', '后', '参加', '直言不讳', '角度', '高', '消息', '就让', '真性情', '实锤', '去年', '自然', '拍', '笑', '一个', '劝嫁', '喜欢', '明星', '那种', '年纪', '吃', '会', '追', '86', '放声', '大', '大笑', '上', '说', '过用', '89', '看得出来', '男子汉', '曝', '北方', '网', '完戏', '年', '中', '3', '综艺节目', '回', '俗语', '学霸', '粉丝', '女孩', '同住', '性格', '网友', '酒店', '句'}, '魏大勋': {'世纪', '一五一十', '美丽', '黄磊', '写道', '事', '恋爱', '故事', '两', '月', '去年', '流传', '江湖', '相关', '上', '赞赏', '完戏', '综艺节目', '八', '一条', '酒店', '都', '如今', '精明', '大多', '颜值', '后', '参加', '拍', '茶余饭后', '前辈', '吃', '说', '89', '社交', '日子', '粉丝', '4', '同住', '优秀', '恋情', '生日', '面前', '何炅', '暗恋', '很多', '杨幂', '一对', '19', '竟然', '实锤', '劝嫁', '年纪', '对着', '86', '大', '正是', '曝', '回', '岁', '网友', '日', '不少', '时', '评论', '爱情故事', '下半年', '传出', '冰淇淋', '直言不讳', '两位', '一个', '不用', '喜欢', '网络', '吐露', '网', '年', '3', '还', '12', '结婚', '区', '人'}, '恋情': {'

C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [100]:
import torch.nn.functional as F 
import torch 
import math

In [133]:
min_loss = 0.0001
loss = 100
max_loop = 4
num_loop = 0
d = 0.85

temp_weight_words = np.ones(num_words)
target_weight_words = np.copy(weight_words)

while min_loss < loss and num_loop < max_loop:
    for i, wi in enumerate(wordSet):
        _sum = 0
        if wi in word_connections:
            for wj in word_connections[wi]:
                j = wordSet.index(wj)
                _sum += graph[wi, wj] * target_weight_words[j] / \
                    sum([ graph[wk, wj] for wk in word_connections[wj] ])
            temp_weight_words[i] = abs(1 - d + d * _sum)
            # print(i, target_weight_words[i], "-->", temp_weight_words[i] )
        else:
            print("word_connections不包含：", wi)

    loss = F.mse_loss(torch.from_numpy(target_weight_words), torch.from_numpy(temp_weight_words))
    loss = loss.tolist()
    target_weight_words = np.copy(temp_weight_words)
    num_loop += 1
    print('num_loop', num_loop, 'loss:', loss)
print('完成！')

num_loop 1 loss: 0.030322870636420248
num_loop 2 loss: 12.527752335010955
num_loop 3 loss: 169.21204015575907
num_loop 4 loss: 112240.71712914215
完成！


In [134]:
target_weight_word_list = [(w, target_weight_words[i]) for i, w in enumerate(wordSet)]
target_weight_word_list = sorted(target_weight_word_list, key=lambda x:x[1], reverse=True)

for i, (word, weight) in enumerate(target_weight_word_list):
    if i < 20 or i > len(target_weight_word_list) - 20 :
        print(i, (word, weight))

0 ('套出', 3496.5304789024326)
1 ('大咧咧', 3088.1797127458462)
2 ('魏大勋', 1405.303263509413)
3 ('89', 1307.0140288068776)
4 ('杨幂', 1304.1512143622892)
5 ('性格', 1182.8583322630384)
6 ('找', 924.9466926131109)
7 ('想', 922.9070358677667)
8 ('孤独', 907.7104694914875)
9 ('前辈', 583.4642320205517)
10 ('女朋友', 476.30783377342016)
11 ('消息', 293.85773068453284)
12 ('北方', 154.48313371997708)
13 ('网站', 112.25436312491048)
14 ('很', 109.08550737891709)
15 ('岁', 100.55170627384041)
16 ('不少', 97.59821107076485)
17 ('中', 76.65425301210269)
18 ('更', 72.48450474176643)
19 ('年', 67.37841095316013)
240 ('更是', 0.15701569485677097)
241 ('的确', 0.15647366973124963)
242 ('日', 0.14983524504265905)
243 ('聊聊吧', 0.14648559896232982)
244 ('来说', 0.14364963477800263)
245 ('助理', 0.14182435573097227)
246 ('算是', 0.12825144773576214)
247 ('说', 0.1251336826213443)
248 ('走', 0.12396676739310501)
249 ('事情', 0.12089148770779676)
250 ('史', 0.11913880043043473)
251 ('不', 0.11170859362724711)
252 ('捷径', 0.10722494984238458)
253 ('金砖', 0

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。


3.展示一些你模型的输出例子。
